In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_250391/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_48_0,0.999958,0.827877,0.820337,0.999890,0.000176,1.150989,0.642646,0.000577,0.029112,0.013259,1.002703,0.013467,147.292443,288.627616,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_45_2,0.999949,0.827054,0.819732,0.999779,0.000210,1.156493,0.644809,0.000609,0.027935,0.014502,1.003234,0.014730,146.933849,288.269021,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_48_1,0.999949,0.828577,0.821163,0.999834,0.000211,1.146306,0.639690,0.000871,0.027014,0.014518,1.003241,0.014746,146.929561,288.264734,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_47_9,0.999949,0.827563,0.821129,0.999975,0.000211,1.153084,0.639813,0.000160,0.027565,0.014539,1.003250,0.014768,146.923743,288.258916,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_47_8,0.999949,0.827592,0.821183,0.999977,0.000212,1.152894,0.639617,0.000147,0.026993,0.014551,1.003255,0.014780,146.920439,288.255611,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,model_1_1_3,0.898403,0.721927,0.711870,0.890734,0.422901,1.859475,1.030623,0.310655,1.280763,0.650308,7.502215,0.660550,131.721236,273.056408,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
487,model_1_1_2,0.896396,0.720324,0.709733,0.888666,0.431255,1.870196,1.038267,0.316536,1.293191,0.656700,7.630665,0.667042,131.682111,273.017284,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
488,model_1_1_1,0.894388,0.718690,0.707427,0.886552,0.439615,1.881122,1.046517,0.322545,1.307709,0.663034,7.759195,0.673476,131.643714,272.978887,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
489,model_1_1_0,0.884942,0.711336,0.685920,0.881328,0.478930,1.930294,1.123444,0.337399,1.348082,0.692048,8.363685,0.702947,131.472401,272.807573,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
